In [1]:
import basis_set_exchange as bse
import scipy as sp
import matplotlib.pyplot as plt
from pyscf import gto,scf
import copy
import numpy as np
import scipy
from scipy.interpolate import interp1d
from pyscf.data.elements import _symbol
from pyscf.grad import RHF as g

In [2]:
import inspect
import sys
sys.path.append("../Libs")
from FcMole import FcM
from FDcoeffs import get_coeffs

In [3]:
fd1=np.asarray([1/12,-2/3 ,0,2/3,-1/12])
fd2=np.asarray([-1/12,4/3,-5/2,4/3,-1/12])
fd3=np.asarray([-1/2,1,0,-1,1/2])
fd4=np.asarray([1,-4,6,-4,1])
fds=[fd1,fd2,fd3,fd4]
fds5=get_coeffs(5)
fds7=get_coeffs(7)

In [4]:
def pcX(z,ref=None):
    if z <1.e-10: raise Exception("non implemented Zero charge atom")
    if z%1 <1.e-10:
        return gto.basis.load(bse.get_basis("pcX-1",fmt="nwchem",elements=[int(z)]),_symbol(int(z)))
    elif ref:
        rb=pcX(ref)
        rbm=pcX(ref-1)
        rbp=pcX(ref+1)
        fb=copy.deepcopy(rb)
        for i in range(len(fb)):  # ref+a(z-zr) +b(z-zr)^2
            a=(rbp[i][1][0]-rbm[i][1][0])/2
            b=(rbp[i][1][0]+rbm[i][1][0]-2*rb[i][1][0])
            fb[i][1][0]+=a*(z-ref)+b*(z-ref)**2

        return fb
    else: raise Exception("can't get the fractional charge basis set, frac val and no ref")

The formula for the gradient is stated in Pople's article (Eq.21) as: 
$$ \frac{\partial E}{\partial Z}= \sum_{\mu\nu}P_{\mu\nu}\frac{\partial H_{\mu\nu}}{\partial Z}+\frac{1}{2}\sum_{\mu\nu\lambda\sigma}
P_{\mu\nu}P_{\lambda\sigma}\frac{\partial}{\partial Z}(\mu \lambda | | \nu\sigma)+\frac{\partial V_{nuc}}{\partial Z} 
-\sum_{\mu\nu}W_{\mu\nu}\frac{\partial S_{\mu\nu}}{\partial Z}
$$
$W$ is an energy weighted density matrix:
$$ W_{\mu\nu}= \sum_i ^{mo.occ.} \epsilon_i c_{\mu i} c_{\nu i}^\dagger
$$

In [13]:
print(inspect.getsource(g.kernel))
print(inspect.getsource(g.grad_elec))
print(inspect.getsource(g.hcore_generator))

    def kernel(self, mo_energy=None, mo_coeff=None, mo_occ=None, atmlst=None):
        cput0 = (time.clock(), time.time())
        if mo_energy is None: mo_energy = self.base.mo_energy
        if mo_coeff is None: mo_coeff = self.base.mo_coeff
        if mo_occ is None: mo_occ = self.base.mo_occ
        if atmlst is None:
            atmlst = self.atmlst
        else:
            self.atmlst = atmlst

        if self.verbose >= logger.WARN:
            self.check_sanity()
        if self.verbose >= logger.INFO:
            self.dump_flags()

        de = self.grad_elec(mo_energy, mo_coeff, mo_occ, atmlst)
        self.de = de + self.grad_nuc(atmlst=atmlst)
        if self.mol.symmetry:
            self.de = self.symmetrize(self.de, atmlst)
        logger.timer(self, 'SCF gradients', *cput0)
        self._finalize()
        return self.de

def grad_elec(mf_grad, mo_energy=None, mo_coeff=None, mo_occ=None, atmlst=None):
    '''
    Electronic part of RHF/RKS gradients

    Args:
        

In [60]:
nn=gto.M(atom="N 0 0 0; N 0 0 2.05",unit="Bohr",basis=bse.get_basis("pcX-1",fmt="nwchem",elements=[6,7,8]))
mf=scf.RHF(nn)
e_nn=mf.scf()

converged SCF energy = -108.916591616523


In [40]:
S=mf.get_ovlp()
h1=mf.get_hcore()
P=mf.make_rdm1()
C=mf.mo_coeff
O=mf.mo_occ
e=mf.mo_energy

In [18]:
g_ijkl=nn.intor("int2e_sph")

In [46]:
cn=gto.M(atom="C 0 0 0; N 0 0 2.05",unit="Bohr",basis=bse.get_basis("pcX-1",fmt="nwchem",elements=[6,7,8]),charge=-1)
on=gto.M(atom="O 0 0 0; N 0 0 2.05",unit="Bohr",basis=bse.get_basis("pcX-1",fmt="nwchem",elements=[6,7,8]),charge=1)

In [64]:
mf_on=scf.RHF(on)
mf_cn=scf.RHF(cn)
e_on=mf_on.scf()
e_cn=mf_cn.scf()

converged SCF energy = -128.874363294639
converged SCF energy = -92.2576688695164


In [75]:
dS=(on.get_ovlp()-cn.get_ovlp())/2
dh1=(on.get_hcore()-cn.get_hcore())/2
dG=(on.intor("int2e_sph")-cn.intor("int2e_sph"))/2
dVnn=(8*7-7*7)/2.05

converged SCF energy = -128.874363294639
converged SCF energy = -92.2576688695166
converged SCF energy = -128.874363294639
converged SCF energy = -92.2576688695165


In [76]:
print(inspect.getsource(mf.Gradients().make_rdm1e))

    def make_rdm1e(self, mo_energy=None, mo_coeff=None, mo_occ=None):
        if mo_energy is None: mo_energy = self.base.mo_energy
        if mo_coeff is None: mo_coeff = self.base.mo_coeff
        if mo_occ is None: mo_occ = self.base.mo_occ
        return make_rdm1e(mo_energy, mo_coeff, mo_occ)



In [77]:
#from np.grad.rhf   , makes w
def make_rdm1e(mo_energy, mo_coeff, mo_occ):
    '''Energy weighted density matrix'''
    mo0 = mo_coeff[:,mo_occ>0]
    mo0e = mo0 * (mo_energy[mo_occ>0] * mo_occ[mo_occ>0])
    return np.dot(mo0e, mo0.T.conj())

In [78]:
W=make_rdm1e(e,C,O)

$$ \frac{\partial E}{\partial Z}= \sum_{\mu\nu}P_{\mu\nu}\frac{\partial H_{\mu\nu}}{\partial Z}+\frac{1}{2}\sum_{\mu\nu\lambda\sigma}
P_{\mu\nu}P_{\lambda\sigma}\frac{\partial}{\partial Z}(\mu \lambda | | \nu\sigma)+\frac{\partial V_{nuc}}{\partial Z} 
-\sum_{\mu\nu}W_{\mu\nu}\frac{\partial S_{\mu\nu}}{\partial Z}
$$
$W$ is an energy weighted density matrix:
$$ W_{\mu\nu}= \sum_i ^{mo.occ.} \epsilon_i c_{\mu i} c_{\nu i}^\dagger
$$

In [79]:
np.einsum("ij,ij",P,dh1),0.5*np.einsum("ij,kl,ijkl",P,P,dG) ,dVnn,-np.einsum("ij,ij",W,dS)

(-23.649717805619975,
 2.793744776549456,
 3.414634146341464,
 -0.06667508592367628)

In [80]:
e_nn,np.einsum("ij,ij",P,dh1)+0.5*np.einsum("ij,kl,ijkl",P,P,dG) +dVnn-np.einsum("ij,ij",W,dS),\
e_nn+np.einsum("ij,ij",P,dh1)+0.5*np.einsum("ij,kl,ijkl",P,P,dG) +dVnn-np.einsum("ij,ij",W,dS)

(-108.91659161652322, -17.508013968652733, -126.42460558517595)

In [81]:
e_nn-np.einsum("ij,ij",P,dh1)-0.5*np.einsum("ij,kl,ijkl",P,P,dG) -dVnn+np.einsum("ij,ij",W,dS)

-91.4085776478705

In [82]:
e_on,e_cn

(-128.87436329463884, -92.25766886951644)

In [69]:
def E_cn(l):
    mol_l=FcM(fcs=[l,0],atom="N1 0 0 0; N2 0 0 2.05",unit="Bohrs",basis={"N1":pcX(7+l,ref=7),"N2":pcX(7)},verbose=1)
    mf_l=scf.RHF(mol_l)
    #mf_l.conv_tol=1e-12
    #mf_l.conv_tol_grad=1e-12
    e=mf_l.scf(dm0=mf_l.init_guess_by_1e())
    return e